<a href="https://colab.research.google.com/github/stan-dev/example-models/blob/master/jupyter/covid-inf-rate/SantaClara_CmdStanPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Bayesian analysis inference of coronavirus infection rate from the Stanford study in Santa Clara county

This Google Colab Python notebook provides the models and data discussed here:  [Simple Bayesian analysis inference of coronavirus infection rate from the Stanford study in Santa Clara county](https://statmodeling.stat.columbia.edu/2020/05/01/simple-bayesian-analysis-inference-of-coronavirus-infection-rate-from-the-stanford-study-in-santa-clara-county/)

It uses CmdStanPy to compile and fit the model.  


#### First up, a little admin to get the latest CmdStanPy and CmdStan installed on this instance.


In [0]:
# Load packages used in this notebook
import os
import json
import shutil
import urllib.request
import pandas as pd

In [0]:
# Please use the latest version of CmdStanPy
pip install --upgrade cmdstanpy

In [0]:
# Install pre-built CmdStan binary
# (faster than compiling from source via install_cmdstan() function)
tgz_file = 'colab-cmdstan-2.23.0.tar.gz'
tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz'
if not os.path.exists(tgz_file):
    urllib.request.urlretrieve(tgz_url, tgz_file)
    shutil.unpack_archive(tgz_file)

In [0]:
# Specify CmdStan location via environment variable
os.environ['CMDSTAN'] = './cmdstan-2.23.0'
# Check CmdStan path
from cmdstanpy import CmdStanModel, cmdstan_path
cmdstan_path()

#### Upload models and data from GitHub

The models and data are available on the Stan GitHub repo https://github.com/stan-dev/example-models/tree/master/jupyter/covid-inf-rate

In [0]:
# Upload models and data onto this instance (raw github content files)
urllib.request.urlretrieve('https://raw.githubusercontent.com/stan-dev/example-models/master/jupyter/covid-inf-rate/data/santa_clara_all.data.json', 'santa_clara_all.data.json')
urllib.request.urlretrieve('https://raw.githubusercontent.com/stan-dev/example-models/master/jupyter/covid-inf-rate/data/santa_clara_apr_11.data.json', 'santa_clara_apr_11.data.json')
urllib.request.urlretrieve('https://raw.githubusercontent.com/stan-dev/example-models/master/jupyter/covid-inf-rate/data/santa_clara_apr_27.data.json', 'santa_clara_apr_27.data.json')
urllib.request.urlretrieve('https://raw.githubusercontent.com/stan-dev/example-models/master/jupyter/covid-inf-rate/stan/pool_sens_spec.stan', 'pool_sens_spec.stan')
urllib.request.urlretrieve('https://raw.githubusercontent.com/stan-dev/example-models/master/jupyter/covid-inf-rate/stan/hier_sens_spec.stan', 'hier_sens_spec.stan')
urllib.request.urlretrieve('https://raw.githubusercontent.com/stan-dev/example-models/master/jupyter/covid-inf-rate/stan/hier_sens_spec_offset_mult.stan', 'hier_sens_spec_offset_mult.stan')
!ls
!echo ""
!echo "santa_clara_apr_11.data.json:"
!cat santa_clara_apr_11.data.json

In the Santa Clara study, of the 3330 people tested (`n_sample`), 50 test results were positive (`y_sample`).  The first version of the paper was released on April 11th, and used specificity data from two studies and sensitivity data from two other studies.

In [0]:
apr_11 = {}
with open('santa_clara_apr_11.data.json') as json_file:
    apr_11 = json.load(json_file)
apr_11

### Simple Model

The first Stan model in this blogpost is called "santaclara.stan".  It a complete pooling model, therefore we'll call it "pool_sens_spec.stan".

In [0]:
!cat pool_sens_spec.stan

We use CmdStanPy to compile and fit the model to the data.

In [0]:
pool_model = CmdStanModel(stan_file='pool_sens_spec.stan')
apr_11_fit = pool_model.sample(data=apr_11)

CmdStanPy reports the central 90% interval, (following McElreath, 2016).  Given the April 11th dataset, this interval is [0.002 - 0.018], thus the data are consistent with an underlying infection rate of between 0 and 2%.

In [0]:
apr_11_fit.summary().round(decimals=3).iloc[1:4,:]

In [0]:
apr_11_drawset = apr_11_fit.get_drawset()
apr_11_drawset.p.plot.hist()

We repeat this procedure with the data from the April 27th version of the study, which has specificity data from 13 studies and sensitivity data from 3 studies.  Again, we pool the results from all studies.

In [0]:
apr_27 = {}
with open('santa_clara_apr_11.data.json') as json_file:
    apr_27 = json.load(json_file)
apr_27

In [0]:
apr_27_fit = pool_model.sample(data=apr_27)
apr_27_fit.summary().round(decimals=3).iloc[1:4,:]


In [0]:
apr_27_drawset = apr_27_fit.get_drawset()
apr_27_drawset.p.plot.hist()

### Hierarchical Model

The hierarchical model allows the sensitivities and specificities to vary across studies.
In the blogpost is called "santaclara_hier.stan".  We'll call it "hier_sens_spec.stan"

In [0]:
!cat hier_sens_spec.stan

__Note__: this model has the suggested tight prior on `sigma_sens` in order to compensate for only having data from 3 studies to fit.

For this model, the data is broken out by study.  We copied this data out of page 19 of the report and munged it into the forms required by the model's data block variable definitions.

In [0]:
hier_data = {}
with open('santa_clara_all.data.json') as json_file:
    hier_data = json.load(json_file)
hier_data

In order to avoid divergent iterations, which result in biased estimates, we set the parameter adapt_delta to 0.98.  To check the resulting sample, we run CmdStan's `diagnose` utility.

In [0]:
hier_model = CmdStanModel(stan_file='hier_sens_spec.stan')
hier_fit = hier_model.sample(data=hier_data, adapt_delta=0.98)

In [0]:
hier_fit.diagnose()

In [0]:
hier_fit.summary().loc[['p', 'mu_spec', 'sigma_spec', 'mu_sens', 'sigma_sens'],].round(decimals=3)

This central 90% interval is [0.005 - 0.022].

In [0]:
hier_drawset = hier_fit.get_drawset()
hier_drawset.p.plot.hist(bins=20,range=(0.0,0.03),density=True)

Bob Carpenter provided a second version of the hierarchical model in which the parameters for the item-level specificities and sensitivies are specified using the Stan language "offset, multiplier" syntax which allows for affine transforms on real-valued variables.  See the Stan Language Reference Manual, chapter [Univariate Data Types and Variable Declarations](https://mc-stan.org/docs/2_23/reference-manual/univariate-data-types-and-variable-declarations.html) for further details. We call this model "hier_sens_spec_offset_mult.stan"


In [0]:
!cat hier_sens_spec_offset_mult.stan

In [0]:
hier_model_v2 = CmdStanModel(stan_file='hier_sens_spec_offset_mult.stan')
hier_fit_v2 = hier_model_v2.sample(data=hier_data, adapt_delta=0.98)
hier_fit_v2.diagnose()

In [0]:
hier_fit_v2.summary().loc[['p', 'mu_logit_spec', 'sigma_logit_spec', 'mu_logit_sens', 'sigma_logit_sens'],].round(decimals=3)

In [0]:
hier_drawset_v2 = hier_fit_v2.get_drawset()
hier_drawset_v2.p.plot.hist(bins=20,range=(0.0,0.03),density=True)

### Play around with the models and data

If you want to play around with your models and data, you can do so using Colab's `file.upload` utility.

In [0]:
from google.colab import files

The `files.upload()` function allows you to choose files on your machine to upload. It returns a Python dictionary which is a map of filenames to their contents.

In [0]:
uploaded = files.upload()
uploaded

The uploaded files are also saved into the current working directory, so you can refer to them directly by filename. Therefore, once you've uploaded a new Stan program file, you can use this colab notebook to fit your model to the data.

In [0]:
!ls -lt | head -3